# HW2 - Q4: Training a robust model & Tighter Certification (10 points)

**Keywords**: Adversarial Robustness Training

**About the dataset**: \
The [MNIST](https://en.wikipedia.org/wiki/MNIST_database) database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems.\
The MNIST database contains 70,000 labeled images. Each datapoint is a $28\times 28$ pixels grayscale image.

**Agenda**:
* In this programming challenge, you will train a 2-hidden layer neural network which is robust to adversarial attacks. 
* You will train models on adversarial examples generated using both FGSM and PGD.
* Finally, you will compare the robustness of a standard (non-robust) model vs. the robust models using both IBP and FastLin bound Algorithms.


**Note:**
* **It is recommended that you use GPU hardware accelaration for this question.**
* A note on working with GPU:
  * Take care that whenever declaring new tensors, set `device=device` in parameters. 
  * You can also move a declared torch tensor/model to device using `.to(device)`. 
  * To move a torch model/tensor to cpu, use `.to('cpu')`
  * Keep in mind that all the tensors/model involved in a computation have to be on the same device (CPU/GPU).
* Run all the cells in order.
* **Do not edit** the cells marked with !!DO NOT EDIT!!
* Only **add your code** to cells marked with !!!! YOUR CODE HERE !!!!
* Do not change variable names, and use the names which are suggested.



---



---



## Preprocessing:

In [49]:
# install this library
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
# !!DO NOT EDIT!!
# imports 
import torch
import torch.nn as nn
import numpy as np
import requests
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import gdown
from zipfile import ZipFile

# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# loading the dataset full MNIST dataset
mnist_train = datasets.MNIST("./data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("./data", train=False, download=True, transform=transforms.ToTensor())

mnist_train.data = mnist_train.data.to(device)
mnist_test.data = mnist_test.data.to(device)

mnist_train.targets = mnist_train.targets.to(device)
mnist_test.targets = mnist_test.targets.to(device)

# reshape and min-max scale
X_train =  (mnist_train.data.reshape((mnist_train.data.shape[0], -1))/255).to(device)
y_train = mnist_train.targets
X_test = (mnist_test.data.reshape((mnist_test.data.shape[0], -1))/255).to(device)
y_test = mnist_test.targets

# load pretrained and dummy model
url_nn_model = 'https://bit.ly/3sKvyOs'
url_models   = 'https://bit.ly/3lsVcDn'
gdown.download(url_nn_model, 'nn_model.pt')
gdown.download(url_models, 'models.zip')
ZipFile("models.zip").extractall("./")

from model import NN_Model
from test_model import Test_Model
nn_model = torch.load("./nn_model.pt").to(device)
print('Pretrained model (nn_model):', nn_model)

test_model = Test_Model()
print('Dummy model (test_model):', test_model)

# This will save the linear layers of the neural network model in a ordered list
# Eg:
# to access weight of first layer: model_layers[0].weight
# to access bias of first layer: model_layers[0].bias
model_layers = [layer for layer in nn_model.children()] # for nn_model
test_model_layers = [layer for layer in test_model.children()] # for dummy model

# first few examples
example_data = mnist_test.data[:18]/255
example_data_flattened  = example_data.view((example_data.shape[0], -1)).to(device) # needed for training
example_labels = mnist_test.targets[:18].to(device)

Downloading...
From: https://bit.ly/3sKvyOs
To: /content/nn_model.pt
100%|██████████| 7.46M/7.46M [00:00<00:00, 154MB/s]
Downloading...
From: https://bit.ly/3lsVcDn
To: /content/models.zip
100%|██████████| 1.55k/1.55k [00:00<00:00, 4.62MB/s]

Pretrained model (nn_model): NN_Model(
  (l1): Linear(in_features=784, out_features=1024, bias=True)
  (l2): Linear(in_features=1024, out_features=1024, bias=True)
  (l3): Linear(in_features=1024, out_features=10, bias=True)
)
Dummy model (test_model): Test_Model(
  (l1): Linear(in_features=2, out_features=3, bias=True)
  (l2): Linear(in_features=3, out_features=3, bias=True)
  (l3): Linear(in_features=3, out_features=2, bias=True)
)




---



---




### **Adversarial training:** In order to train robust models, the most intuitive strategy is to train on adversarial examples.
* The adversarial (robust) loss function is defined as:
$\underset{\theta}{minimize} \frac{1}{|S|}\sum_{x,y\in S} \underset{∥δ∥≤ϵ}{max}\,ℓ(h_θ(x+δ),y)$, \
where $\theta$ are the learnable parameters, $S$ is the set of training examples with $x$ representing the input example and $y$ the ground truth label, $h_\theta$ is the hypothesis (neural network model), $\delta$ is the attack perturbation, and $\epsilon$ is the attack budget.

* This is also known as the min-max loss function. The gradient descent step now becomes:\
$\theta:=\theta-\frac{\alpha}{|B|}\sum_{x,y\in B} ∇_θ \underset{∥δ∥≤ϵ}{max}\,ℓ(h_θ(x+δ),y)$,\
where $B$ is the mini-batch and $\alpha$ is the learning rate. 

* Now the question becomes how to find the inner term: $∇_θ \underset{∥δ∥≤ϵ}{max}\,ℓ(h_θ(x+δ),y)$ of the gradient descent step. For this, we can use **Danskin’s Theorem**, which states that to compute the (sub)gradient of a function containing a max term, we need to simply 1) find the maximum, and 2) compute the normal gradient evaluated at this point. It holds only when you have the exact maximum.  Note that it is not possible to solve the inner maximization problem exactly (NP-hard). However, the better job we do of solving the inner maximization problem, the closer it seems that Danskin’s theorem starts to hold. That is why we can re-use methods such as FGSM/PGD to find approximate worst case examples. In other words, we can perform the attack to find $δ^{*} = \underset{∥δ∥≤ϵ}{argmax}ℓ(h_θ(x+δ),y)$, and then compute this term at the perturbed image: $∇_θℓ(h_θ(x+δ^{*}),y)$.

In summary, we would create an adversarial example for each datapoint in the mini-batch and add the loss corresponding to that example to the gradient.  



---



### **(a) Setup:** (2 points)
### **#1.**Get the attack functions `fgsm` and `pgd` that you defined in the HW2-Q3 (a) and (b). Modify the `pgd` function to start with random values of `delta` instead of zeros.

In [51]:
#######
# !!! YOUR CODE HERE !!!
import torch.nn as nn
def fgsm(model, X, y, epsilon = 0.05):
    # Setting tensor parameters
    X = X.to(device)
    y = y.to(device)
    X.requires_grad = True

    # setting initial perturbation tensor
    perturbation = torch.zeros_like(X, requires_grad=True)

    # reset gradients
    model.zero_grad()

    # getting predictions
    y_pred = model(X + perturbation)

    # calculate loss
    criterion = nn.CrossEntropyLoss()
    loss = criterion(y_pred, y).to(device)

    # backpropagate loss
    loss.backward()

    # calculate perturbation
    perturbation = epsilon * (perturbation.grad.sign())
    
    return perturbation.detach()

def pgd(model, X, y, alpha = 1000, epsilon = 0.05, num_iter = 1000):
    
    # Setting tensor parameters
    X = X.to(device)
    y = y.to(device)

    # setting criterion for loss function
    criterion = nn.CrossEntropyLoss()

    # Setting initial random values of perturbation tensor
    perturbation = torch.randn_like(X, requires_grad=True)

    for i in range(num_iter):
        X.requires_grad = True

        # prediction 
        y_pred = model(X + perturbation)

        # calculate loss
        loss = criterion(y_pred, y).to(device)

        # backpropagate loss
        loss.backward()

        # calculate perturbation
        perturbation.data = torch.clamp((perturbation + X.shape[0]*alpha*perturbation.grad.data), min = -epsilon, max = epsilon)

        # reset gradient
        perturbation.grad.zero_()

    return perturbation.detach()


#######

### **#2.** Create a 2-hidden-layer neural network model in pytorch. The input should be the size of the flattened MNIST image, and output layer should be of size 10, which is the number of target labels. Each of the two hidden layers should be of size 1024 with ReLU activations between each subsequent layer except the last layer.

You can refer to the structure of `nn_model` from HW2-Q3. It should be similar to that.

In [52]:
#######
# !!!! YOUR CODE HERE !!!!
num_features = X_train.shape[1]
num_classes = 10
class NN_Model(torch.nn.Module):
  def __init__(self):
    super(NN_Model, self).__init__()
    self.linear1 = nn.Linear(num_features, 1024)
    self.linear2 = nn.Linear(1024, 1024)
    self.linear3 = nn.Linear(1024, num_classes)

    self.relu = nn.ReLU()    

  def forward(self, X):
    activation1 = self.relu(self.linear1(X))
    activation2 = self.relu(self.linear2(activation1))
    output = self.linear3(activation2)
    return output
#######

### **#3.** Define a function `train_torch_model_adversarial` that takes as input an initialized torch model (`model`), batch size (`batch_size`), initialized loss (`criterion`), max number of epochs (`max_epochs`), training data (`X_train, y_train`), learning rate (`lr`), tolerance for stopping (`tolerance`), adversarial strategy (`adversarial_strategy`: `None/'fgsm'/'pgd'`), and attack budget(`epsilon`). 

### **Note**: 
* If `adversarial_strategy` is `None`, don't train on adversarial examples.
* This function will return a tuple of `(model, losses)`, where `model` is the trained model, and `losses` are a list of tuple of loss logged every epoch. 
* The only difference from the function `train_torch_model` that you wrote in HW2-Q2 (b) is that you also add gradient of adversarial example. 

In [53]:
#######
# !!!! YOUR CODE HERE !!!!
# Define a function train_torch_model
import math

def train_torch_model(model, batch_size, criterion, max_epochs, X_train, y_train, lr, tolerance, adversarial_strategy, epsilon):
  losses = []
  prev_loss = float('inf')
  number_of_batches = math.ceil(len(X_train)/batch_size)
  
  #######
  # !!!! YOUR CODE HERE !!!!
  # 3. move model to device
  model = model.to(device)

  # 4. define optimizer (use torch.optim.SGD (Stochastic Gradient Descent)) 
  # Set learning rate to lr and also set model parameters
  optimizer = torch.optim.SGD(model.parameters(), lr = lr) 

  for epoch in tqdm(range(max_epochs)):
    for i in range(number_of_batches):
      X_train_batch = X_train[i*batch_size: (i+1)*batch_size]
      y_train_batch = y_train[i*batch_size: (i+1)*batch_size]

      # 5. reset gradients
      optimizer.zero_grad()

      if adversarial_strategy == None:
        # 6. prediction
        y_pred_batch = model(X_train_batch)

        # 7. calculate loss
        loss = criterion(y_pred_batch, y_train_batch)
      
      if adversarial_strategy == 'fgsm':
        # 6. prediction
        delta = fgsm(model, X_train_batch, y_train_batch, epsilon = 0.05)
        y_pred_batch = model(X_train_batch + delta)
  
        # 7. calculate loss
        loss = criterion(y_pred_batch, y_train_batch)

      if adversarial_strategy == 'pgd':
        # 6. prediction
        delta = pgd(model, X_train_batch, y_train_batch, alpha = 0.01, epsilon = 0.05, num_iter = 100)
        y_pred_batch = model(X_train_batch + delta)
  
        # 7. calculate loss
        loss = criterion(y_pred_batch, y_train_batch)

      # 8. backpropagate loss
      loss.backward()

      # 9. perform a single gradient update step
      optimizer.step()

  #######

    # log loss every epoch and print every epoch:
    losses.append((epoch, loss.item()))
    print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))
    
    # break if decrease in loss is less than threshold
    if abs(prev_loss-loss)<=tolerance:
      break
    else:
      prev_loss=loss  

  # return updated model and logged losses
  return model, losses
#######



---



### **(b) Train the model with different strategies:** (3 points)
###  **#1.** Train three models, one without adversarial training, one with adversarial training using `fgsm`, and last one with adversarial training using `pgd`. 

* Use attack budget `epsilon` of 0.05.

* Use a mini-batch of size 512, and train for 20 epochs with learning rate $10^{-2}$, and early stopping tolerance of $10^{-6}$. Report the loss in each case.

* For `pgd`, use the value of step-size `alpha=0.01` and number of iterations `num_iter=100` when training.

In [54]:
#######
# !!!! YOUR CODE HERE !!!!
model = NN_Model()
criterion = nn.CrossEntropyLoss()

print('Normal Model')
model_normal, losses_normal = train_torch_model(model, 512, criterion, 20, X_train, y_train, lr = 0.01, tolerance = 1e-6,
                                  adversarial_strategy=None, epsilon=0.05)
print('Final Loss = ', losses_normal[-1])
print('----------------------------------------------')

print('FGSM Model')
model_fgsm, losses_fgsm = train_torch_model(model, 512, criterion, 20, X_train, y_train, lr = 0.01, tolerance = 1e-6,
                                  adversarial_strategy='fgsm', epsilon=0.05)
print('Final Loss = ', losses_fgsm[-1])
print('----------------------------------------------')


print('PGD Model')
model_pgd, losses_pgd = train_torch_model(model, 512, criterion, 20, X_train, y_train, lr = 0.01, tolerance = 1e-6,
                                  adversarial_strategy='pgd', epsilon=0.05)
print('Final Loss = ', losses_pgd[-1])
print('----------------------------------------------')


#######

Normal Model


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0, Loss: 2.2338333129882812
Epoch: 1, Loss: 2.1168181896209717
Epoch: 2, Loss: 1.8835252523422241
Epoch: 3, Loss: 1.5130053758621216
Epoch: 4, Loss: 1.162898302078247
Epoch: 5, Loss: 0.9281675815582275
Epoch: 6, Loss: 0.7788661122322083
Epoch: 7, Loss: 0.6804671287536621
Epoch: 8, Loss: 0.6123895049095154
Epoch: 9, Loss: 0.5631710886955261
Epoch: 10, Loss: 0.5262160897254944
Epoch: 11, Loss: 0.49744927883148193
Epoch: 12, Loss: 0.47444844245910645
Epoch: 13, Loss: 0.4555703103542328
Epoch: 14, Loss: 0.43976831436157227
Epoch: 15, Loss: 0.42627251148223877
Epoch: 16, Loss: 0.4145399034023285
Epoch: 17, Loss: 0.40415456891059875
Epoch: 18, Loss: 0.3948976993560791
Epoch: 19, Loss: 0.3865472078323364
Final Loss =  (19, 0.3865472078323364)
----------------------------------------------
FGSM Model


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0, Loss: 1.1881651878356934
Epoch: 1, Loss: 1.085912823677063
Epoch: 2, Loss: 1.022955298423767
Epoch: 3, Loss: 0.9800436496734619
Epoch: 4, Loss: 0.9480516910552979
Epoch: 5, Loss: 0.9231178164482117
Epoch: 6, Loss: 0.902167022228241
Epoch: 7, Loss: 0.8834124207496643
Epoch: 8, Loss: 0.8676431775093079
Epoch: 9, Loss: 0.8519196510314941
Epoch: 10, Loss: 0.8365588188171387
Epoch: 11, Loss: 0.8221160769462585
Epoch: 12, Loss: 0.8092756867408752
Epoch: 13, Loss: 0.7955482602119446
Epoch: 14, Loss: 0.7831981778144836
Epoch: 15, Loss: 0.7711530327796936
Epoch: 16, Loss: 0.7588655948638916
Epoch: 17, Loss: 0.7472127079963684
Epoch: 18, Loss: 0.7352306842803955
Epoch: 19, Loss: 0.7243234515190125
Final Loss =  (19, 0.7243234515190125)
----------------------------------------------
PGD Model


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0, Loss: 1.549898624420166
Epoch: 1, Loss: 0.7431626915931702
Epoch: 2, Loss: 0.4415706396102905
Epoch: 3, Loss: 0.3323846757411957
Epoch: 4, Loss: 0.2652454376220703
Epoch: 5, Loss: 0.2116582989692688
Epoch: 6, Loss: 0.19527240097522736
Epoch: 7, Loss: 0.1882706731557846
Epoch: 8, Loss: 0.17313772439956665
Epoch: 9, Loss: 0.1783478856086731
Epoch: 10, Loss: 0.1602720469236374
Epoch: 11, Loss: 0.15520521998405457
Epoch: 12, Loss: 0.13474538922309875
Epoch: 13, Loss: 0.1261933594942093
Epoch: 14, Loss: 0.11934738606214523
Epoch: 15, Loss: 0.11121489852666855
Epoch: 16, Loss: 0.09578990191221237
Epoch: 17, Loss: 0.08479103446006775
Epoch: 18, Loss: 0.09344472736120224
Epoch: 19, Loss: 0.09549833089113235
Final Loss =  (19, 0.09549833089113235)
----------------------------------------------




---



### **#2.**Measuring performance: In this part we will be evaluate the trained models for different test scenarios. 


### Print the accuracy of each of the three trained models on the clean test dataset. 

In [55]:
#######
# !!!! YOUR CODE HERE !!!!
from sklearn.metrics import accuracy_score

def get_accuracy(model, X_test_torch, y_test):
  predictions_test = model(X_test_torch).to('cpu')
  y_test_pred = torch.argmax(predictions_test, dim=1).numpy()
  y_test = y_test.to('cpu')
  return accuracy_score(y_test_pred, np.asarray(y_test, dtype=np.float32))

accuracy_normal = get_accuracy(model_normal, X_test, y_test)
print("Normal Model Accuracy = ", accuracy_normal)

accuracy_fgsm = get_accuracy(model_fgsm, X_test, y_test)
print("FGSM Model Accuracy = ", accuracy_fgsm)

accuracy_pgd = get_accuracy(model_pgd, X_test, y_test)
print("PGD Model Accuracy = ", accuracy_pgd)

#######

Normal Model Accuracy =  0.9778
FGSM Model Accuracy =  0.9778
PGD Model Accuracy =  0.9778


### **#3.** Using the same test dataset, perform adversarial attack to compute robust accuracy for each of the three models. Report the robust accuracy of each model for both:
###(a) FGSM attack
###(b) PGD attack

### To create PGD attack examples, use `alpha=0.01`, `num_iter=100`.

In [56]:
#######
# !!!! YOUR CODE HERE !!!!

delta_fgsm = fgsm(model, X_test, y_test, epsilon = 0.05)

delta_pgd = pgd(model, X_test, y_test, alpha = 0.01, epsilon = 0.05, num_iter = 100)

test_accuracy_fgsm = get_accuracy(model_fgsm, X_test + delta_fgsm, y_test)
print("FGSM Model Robust Accuracy = ", test_accuracy_fgsm)

test_accuracy_pgd = get_accuracy(model_pgd, X_test + delta_pgd, y_test)
print("PGD Model Robust Accuracy = ", test_accuracy_pgd)


#######

FGSM Model Robust Accuracy =  0.843
PGD Model Robust Accuracy =  0.9168




---



---



### **(c) FastLin Bound Propagation Algorithm:** In this part, we will use Fast Linear (Fast-Lin) algorithm to get lower and upper bounds for each layer of the neural network. (2.5 points)

**Note:** See Section 3.3 and algorithm 1 in the appendix (Section D: Algorithms) from this paper: \
[Weng etal, Towards Fast Computation of Certified Robustness for ReLU Networks, ICML 2018](https://arxiv.org/pdf/1804.09699.pdf)


### Define a function `fast_linear_bound` that takes as input an ordered list of neural network model layers (`model_layers`), a single input example (`x`), attack budget (`epsilon`). Consider the $l_\infty$ norm ball for this activity. Return a list of tuples of `pre-activation` lower and upper bound tensors for each layer. 
### Also keep in mind to set device when declaring new Tensors.

In [57]:
#######
# !!! YOUR CODE HERE !!!
def fast_linear_bound(model_layers, x, epsilon):
  pass

#######

In [58]:
# !!DO NOT EDIT!!
sample_epsilon = 0.2
# unit test - 1
x_1 = torch.tensor([[0.1, 0.9]], device=device)
test_bounds_1 = fast_linear_bound(test_model_layers, x_1, sample_epsilon)
assert torch.all(torch.eq(torch.round(test_bounds_1[0][0], decimals=2), torch.tensor([[0.0000, 0.7000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[0][1], decimals=2), torch.tensor([[0.3000, 1.0000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[1][0], decimals=2), torch.tensor([[0.0000, 1.4000, 1.2000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[1][1], decimals=2), torch.tensor([[0.4500, 2.6000, 1.5000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[2][0], decimals=2), torch.tensor([[3.100, -0.5000,  2.4000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[2][1], decimals=2), torch.tensor([[6.25000, -0.2000, 4.0500]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[3][0], decimals=2), torch.tensor([[4.8000, 4.3000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[3][1], decimals=2), torch.tensor([[8.7700, 8.9500]], device=device)))

# unit test - 2
x_2 = torch.tensor([[0.4, 0.5]], device=device)
test_bounds_2 = fast_linear_bound(test_model_layers, x_2, sample_epsilon)
assert torch.all(torch.eq(torch.round(test_bounds_2[0][0], decimals=2), torch.tensor([[0.2000, 0.3000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[0][1], decimals=2), torch.tensor([[0.6000, 0.7000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[1][0], decimals=2), torch.tensor([[0.4000, 1.0000, 0.8000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[1][1], decimals=2), torch.tensor([[1.0000, 2.6000, 1.2000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[2][0], decimals=2), torch.tensor([[0.4000, -0.3000, 0.8000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[2][1], decimals=2), torch.tensor([[4.6000, 0.1000, 3.0000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[3][0], decimals=2), torch.tensor([[1.2300, 0.4300]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[3][1], decimals=2), torch.tensor([[6.7000, 6.8000]], device=device)))

TypeError: ignored



---



### **(d) Comparison of certification strategies** (2.5 points)
### **#1.** Get the functions  `bound_propagation` and `binary_search` that you defined in HW3-Q3(d). Refactor the function `binary_search` so that it takes one additional input parameter: `bound_function` which represents a function that can be used to get the bounds, and modify your implementation of `binary_search` so that it uses the parameter `bound_function` for getting the bounds.

In [70]:
#######
# !!! YOUR CODE HERE !!!
def bound_propagation(model_layers, x, epsilon):
  bounds = []
  l, u = (x - epsilon).clamp(min=0), (x + epsilon).clamp(max=1)
  bounds.append((l, u))
  for layer in model_layers:
    if isinstance(layer, nn.Linear):
    # calculating pre-activation bounds
      l_ = (layer.weight.clamp(min=0) @ l.t() + layer.weight.clamp(max=0) @ u.t() + layer.bias[:,None]).t()
      u_ = (layer.weight.clamp(min=0) @ u.t() + layer.weight.clamp(max=0) @ l.t() + layer.bias[:,None]).t()
    
    bounds.append((l_, u_))
    
    if isinstance(layer, nn.ReLU):
    # calculating post-activation bounds
      l_ = l_.clamp(min=0)
      u_ = u_.clamp(min=0)

    l, u = l_, u_
    
  # print(bounds)
  return bounds

# binary search to find optimum epsilon values
def binary_search(epsilons, model_layers, X, y, num_classes, bond_function):
  epsilon_list = []

  # function to get return whether tested epsilon value satisfies the criteria for tensor 1
  def criteria_1(model_layers, epsilon, X, y, num_classes, bond_function):
    if bond_function == 'ibp':
      bounds = bound_propagation(model_layers, X, epsilon)
    i = 0

    lower_bounds =  bounds[-1][0][i].detach().cpu().numpy()
    upper_bounds = bounds[-1][1][i].detach().cpu().numpy()

    flag = 0
    for k in range(num_classes):
      if y[i] != k:
        criteria_value = lower_bounds[y[i]] - upper_bounds[k] 
        if criteria_value < 0:
          flag = -1
          break
    return flag

  # optimum epsilon value for tensor 1
  start1 = 0
  end1 = len(epsilons) - 1
  flag1 = -1
  while start1 <= end1:
    mid = int((start1 + end1)/ 2)
    if (criteria_1(model_layers, epsilons[mid], X, y, num_classes, bond_function) == 0 
    and criteria_1(model_layers, epsilons[mid + 1], X, y, num_classes, bond_function) == -1):
      epsilon_list.append(epsilons[mid])
      flag1 = 0
      break
    elif (criteria_1(model_layers, epsilons[mid], X, y, num_classes, bond_function) == 0 
          and criteria_1(model_layers, epsilons[mid + 1], X, y, num_classes, bond_function) == 0):
      start1 = mid + 1
    else:
      end1 = mid - 1
  # returning None if value not found
  if flag1 == -1:
    epsilon_list.append(None)

  return epsilon_list

#######

### **#2.** Now, consider the first example from `example_data_flattened`. For this example find the value of certified epsilon (using `binary_search`) using (1) IBP (2) Fast-Lin on the standard model that you trained in part (b). Compare your results.

In [71]:
#######
# !!! YOUR CODE HERE !!!
epsilons = [x/10000 for x in range(1, 10000)]
model_normal_layers = [layer for layer in model_normal.children()]

binary_search(epsilons, model_normal_layers, example_data_flattened[0:1], example_labels[0:1], 10, 'ibp')
#######

[0.0033]

### **#3.** Repeat the same activity as above but use the robust model that you trained on PGD adversarial example. Compare you results.

In [72]:
#######
# !!! YOUR CODE HERE !!!
model_pgd_layers = [layer for layer in model_pgd.children()]

binary_search(epsilons, model_pgd_layers, example_data_flattened[0:1], example_labels[0:1], 10, 'ibp')
# We get the same robustness value for standard and PGD model for ibp.
# We did not implement Fast-Lin
#######

[0.0033]

---
---